# Data Cleaning: Finding Repeated Images

This notebook identifies duplicate images in the `data/train/` folder, both within the same label folder and across different labels.

In [2]:
import hashlib
from pathlib import Path
from collections import defaultdict
from PIL import Image

## Setup Paths

In [3]:
train_dir = Path('data/train')
label_folders = ['healthy', 'rust', 'scab', 'multiple_diseases']

print(f"Looking for images in: {train_dir.absolute()}")
print(f"Label folders: {label_folders}")

Looking for images in: /home/nahuel/programming/ML/plant-pathology/data/train
Label folders: ['healthy', 'rust', 'scab', 'multiple_diseases']


## Method 1: Exact Duplicates (MD5 Hash)

This method finds images that are exactly identical by computing MD5 hash of the file content.

In [4]:
def compute_md5_hash(file_path):
    """Compute MD5 hash of a file."""
    hash_md5 = hashlib.md5()
    with open(file_path, "rb") as f:
        for chunk in iter(lambda: f.read(4096), b""):
            hash_md5.update(chunk)
    return hash_md5.hexdigest()

def find_exact_duplicates(train_dir, label_folders):
    """Find exact duplicate images using MD5 hash."""
    hash_to_files = defaultdict(list)
    
    # Compute hash for each image
    for label in label_folders:
        label_path = train_dir / label
        if not label_path.exists():
            print(f"Warning: {label_path} does not exist")
            continue
            
        for img_file in label_path.iterdir():
            if img_file.is_file() and img_file.suffix.lower() in ['.jpg', '.jpeg', '.png', '.bmp']:
                try:
                    file_hash = compute_md5_hash(img_file)
                    hash_to_files[file_hash].append((label, img_file.name))
                except Exception as e:
                    print(f"Error processing {img_file}: {e}")
    
    # Find duplicates
    duplicates = {h: files for h, files in hash_to_files.items() if len(files) > 1}
    
    return duplicates

print("Finding exact duplicates...")
exact_duplicates = find_exact_duplicates(train_dir, label_folders)
print(f"Found {len(exact_duplicates)} groups of exact duplicates")

Finding exact duplicates...
Found 1 groups of exact duplicates


## Display Exact Duplicates

In [5]:
if exact_duplicates:
    print("\n" + "="*80)
    print("EXACT DUPLICATES FOUND")
    print("="*80)
    
    for idx, (file_hash, files) in enumerate(exact_duplicates.items(), 1):
        print(f"\nGroup {idx} ({len(files)} copies):")
        print(f"  Hash: {file_hash}")
        
        # Group by label
        by_label = defaultdict(list)
        for label, filename in files:
            by_label[label].append(filename)
        
        for label, filenames in by_label.items():
            print(f"  - {label}: {filenames}")
        
        # Check if duplicate exists across different labels
        if len(by_label) > 1:
            print(f"  ⚠️  WARNING: Image appears in multiple labels: {list(by_label.keys())}")
else:
    print("No exact duplicates found!")


EXACT DUPLICATES FOUND

Group 1 (2 copies):
  Hash: 191e6188a9a0e55c88f1a96135a28238
  - scab: ['Train_379.jpg']
  - multiple_diseases: ['Train_1173.jpg']
  ⚠️  WARNING: Image appears in multiple labels: ['scab', 'multiple_diseases']


## Method 2: Perceptual Duplicates (Perceptual Hashing)

This method finds images that are visually similar (e.g., slightly compressed, resized, or modified versions).

In [6]:
import imagehash

def find_perceptual_duplicates(train_dir, label_folders, hash_size=8):
    """Find perceptually similar images using perceptual hashing."""
    hash_to_files = defaultdict(list)
    
    # Compute perceptual hash for each image
    for label in label_folders:
        label_path = train_dir / label
        if not label_path.exists():
            continue
            
        for img_file in label_path.iterdir():
            if img_file.is_file() and img_file.suffix.lower() in ['.jpg', '.jpeg', '.png', '.bmp']:
                try:
                    img = Image.open(img_file)
                    # Use average hash for perceptual similarity
                    img_hash = str(imagehash.average_hash(img, hash_size=hash_size))
                    hash_to_files[img_hash].append((label, img_file.name))
                except Exception as e:
                    print(f"Error processing {img_file}: {e}")
    
    # Find duplicates
    duplicates = {h: files for h, files in hash_to_files.items() if len(files) > 1}
    
    return duplicates

print("Finding perceptual duplicates...")
perceptual_duplicates = find_perceptual_duplicates(train_dir, label_folders)
print(f"Found {len(perceptual_duplicates)} groups of perceptual duplicates")

Finding perceptual duplicates...
Found 5 groups of perceptual duplicates


## Display Perceptual Duplicates

In [7]:
if perceptual_duplicates:
    print("\n" + "="*80)
    print("PERCEPTUAL DUPLICATES FOUND")
    print("="*80)
    print("(These images are visually similar but may not be exactly identical)\n")
    
    for idx, (img_hash, files) in enumerate(perceptual_duplicates.items(), 1):
        print(f"\nGroup {idx} ({len(files)} copies):")
        print(f"  Hash: {img_hash}")
        
        # Group by label
        by_label = defaultdict(list)
        for label, filename in files:
            by_label[label].append(filename)
        
        for label, filenames in by_label.items():
            print(f"  - {label}: {filenames}")
        
        # Check if duplicate exists across different labels
        if len(by_label) > 1:
            print(f"  ⚠️  WARNING: Image appears in multiple labels: {list(by_label.keys())}")
else:
    print("No perceptual duplicates found!")


PERCEPTUAL DUPLICATES FOUND
(These images are visually similar but may not be exactly identical)


Group 1 (2 copies):
  Hash: 1f1f01818381f1f2
  - healthy: ['Train_1817.jpg', 'Train_174.jpg']

Group 2 (2 copies):
  Hash: 3f02010703030307
  - healthy: ['Train_417.jpg', 'Train_1675.jpg']

Group 3 (2 copies):
  Hash: efe7a1e1e3c18111
  - healthy: ['Train_966.jpg', 'Train_616.jpg']

Group 4 (2 copies):
  Hash: edcf87030389c1d1
  - rust: ['Train_828.jpg', 'Train_759.jpg']

Group 5 (2 copies):
  Hash: 78dcd88000301e8f
  - scab: ['Train_379.jpg']
  - multiple_diseases: ['Train_1173.jpg']
  ⚠️  WARNING: Image appears in multiple labels: ['scab', 'multiple_diseases']


## Summary Statistics

In [8]:
# Count total images per label
total_images = defaultdict(int)
for label in label_folders:
    label_path = train_dir / label
    if label_path.exists():
        total_images[label] = len([f for f in label_path.iterdir() 
                                   if f.is_file() and f.suffix.lower() in ['.jpg', '.jpeg', '.png', '.bmp']])

print("\n" + "="*80)
print("SUMMARY")
print("="*80)
print("\nTotal images per label:")
for label, count in total_images.items():
    print(f"  {label}: {count}")

print(f"\nExact duplicates: {len(exact_duplicates)} groups")
print(f"Perceptual duplicates: {len(perceptual_duplicates)} groups")

# Count duplicates across labels
cross_label_exact = sum(1 for files in exact_duplicates.values() 
                        if len(set(label for label, _ in files)) > 1)
cross_label_perceptual = sum(1 for files in perceptual_duplicates.values() 
                             if len(set(label for label, _ in files)) > 1)

print(f"\nDuplicates across different labels:")
print(f"  Exact: {cross_label_exact} groups")
print(f"  Perceptual: {cross_label_perceptual} groups")

# Count duplicates within same label
within_label_exact = len(exact_duplicates) - cross_label_exact
within_label_perceptual = len(perceptual_duplicates) - cross_label_perceptual

print(f"\nDuplicates within same label:")
print(f"  Exact: {within_label_exact} groups")
print(f"  Perceptual: {within_label_perceptual} groups")


SUMMARY

Total images per label:
  healthy: 516
  rust: 622
  scab: 592
  multiple_diseases: 91

Exact duplicates: 1 groups
Perceptual duplicates: 5 groups

Duplicates across different labels:
  Exact: 1 groups
  Perceptual: 1 groups

Duplicates within same label:
  Exact: 0 groups
  Perceptual: 4 groups
